In [4]:
%pip install spacy
%python -m spacy download en_core_web_sm


  Using cached spacy-3.8.4-cp311-cp311-win_amd64.whl.metadata (27 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached murmurhash-1.0.12-cp311-cp311-win_amd64.whl.metadata (2.2 kB)
  Using cached cymem-2.0.11-cp311-cp311-win_amd64.whl.metadata (8.8 kB)
  Using cached preshed-3.0.9-cp311-cp311-win_amd64.whl.metadata (2.2 kB)
  Using cached thinc-8.3.4-cp311-cp311-win_amd64.whl.metadata (15 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached srsly-2.5.1-cp311-cp311-win_amd64.whl.metadata (20 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached typer-0.15.1-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached langcodes-3.5.0-py3-none-any.whl.metadata (29 kB)
  Using cached language_data-1.3.0-py3-no


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [2]:
%pip install pytesseract

  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
Using cached pytesseract-0.3.13-py3-none-any.whl (14 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import cv2
import pytesseract
from PIL import Image
import re
import spacy
import pandas as pd

In [14]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
custom_config = r'--oem 3 --psm 6'

# -----------------------------
# 2. Read and Preprocess the Invoice Image
# -----------------------------
# Replace with the path to your invoice image file
image_path = r"images\rrr.png"  # <-- Change this to your image file path
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"Image not found at {image_path}")

# Convert image to grayscale to improve OCR accuracy
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# Optionally, apply thresholding:
# gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

# (Optional) Save the preprocessed image for debugging
cv2.imwrite("preprocessed_image.jpg", gray)

# -----------------------------
# 3. Extract Text using Tesseract OCR
# -----------------------------
ocr_text = pytesseract.image_to_string(gray, config=custom_config)
print("=== OCR Extracted Text ===")
print(ocr_text)

# -----------------------------
# 4. Inline Clean the Extracted Text
# -----------------------------
# Remove extra spaces and newline characters inline
text = " ".join(ocr_text.split())
print("\n=== Cleaned Text ===")
print(text)

=== OCR Extracted Text ===
“_ | | i | | ll |
8300755602

Invoice Number: 10525RD001820847 PackettD: #200755602
Order Number: 1290196-056445-8121802 Tnvoice Date: 31 Jan 2025,
Nature of Transaction: In-State Order Date: 31 Jan 2025
Place of Supply: UTTAR PRADESH Nature of Supply: Goods
Bl to Ship wo:
Anshika Arya
Bit general store, sadapur Noida Sector 45 Gautam Buddha Customer Type: Unregistered
Nagat - 201303 UP, Indl
Bl From: Ship From:
REDTAPE LIMITED REDTAPE LIMITED
C-4,5, 36,37, Sector $9, Nolda 201301 (Utar Pradesh), Nola, C4, 5, 36. 7, Sector 59, Nolda~ 201301 (Uta Prades), Noida,
ar pradesh 201301 ‘ar pradesh- 201301
GSTIN Number: O9AALCRSO22RIZN

Gross Other Taxable sGsT/
ye Discount Clee ENN cost LSE IGST Coss Ttat Amount
RDTPCASH101503720(R1.L0821-39) - Red Tape Women Round Toe Lace-Ups Sneakers, Size: 6 (UK6)
HSN: 64041990, 9.0% CGST, 9.0% SGSTIUGST,
1 Rs 6899.00 Rs 5658.00 _Rs0,00 Rs 1051.70 Rs 94.65 Rs 94.65 Rs 1241.00
TOTAL Rs 6890.00 Rs565800 _Rs0.00 Rs 1051.70 Rs94.65 

In [15]:
# -----------------------------
# 5. Inline Extraction of Invoice Fields using Regular Expressions
# -----------------------------
invoice_date = re.search(r"Invoice Date:\s*([\d./-]+)", text)
invoice_date = invoice_date.group(1).strip() if invoice_date else "Not Found"

invoice_number = re.search(r"Invoice Number:\s*([\w-]+)", text)
invoice_number = invoice_number.group(1).strip() if invoice_number else "Not Found"

order_number = re.search(r"Order Number:\s*([\w-]+)", text)
order_number = order_number.group(1).strip() if order_number else "Not Found"

seller_name = re.search(r"(?:Sold By|Seller Name):\s*([\w\s,&.]+)", text)
seller_name = seller_name.group(1).strip() if seller_name else "Not Found"

seller_address = re.search(r"(?:Address|Seller Address):\s*([\w\s,.-]+?)(?=\s*(?:PAN No:|GST Registration No:))", text)
seller_address = seller_address.group(1).strip() if seller_address else "Not Found"

buyer_name = re.search(r"(?:Name|Buyer Name):\s*([\w\s]+)", text)
buyer_name = buyer_name.group(1).strip() if buyer_name else "Not Found"

buyer_address = re.search(r"(?:Shipping Address|Buyer Address):\s*([\w\s,.-]+?)(?=\s*(?:Invoice|$))", text)
buyer_address = buyer_address.group(1).strip() if buyer_address else "Not Found"

pan_no = re.search(r"PAN No:\s*([\w]+)", text)
pan_no = pan_no.group(1).strip() if pan_no else "Not Found"

gst_no = re.search(r"GST Registration No:\s*([\w]+)", text)
gst_no = gst_no.group(1).strip() if gst_no else "Not Found"

phone_no = re.search(r"Phone(?: Number)?:\s*(\+?\d[\d\s-]+)", text)
phone_no = phone_no.group(1).strip() if phone_no else "Not Found"

total_amount = re.search(r"Total Amount:\s*([\d,]+\.?\d*)", text)
total_amount = total_amount.group(1).strip() if total_amount else "Not Found"

tax_amount = re.search(r"Tax Amount:\s*([\d,]+\.?\d*)", text)
tax_amount = tax_amount.group(1).strip() if tax_amount else "Not Found"


In [16]:
# 6. Display Extracted Data in a Neat Tabular Format
# -----------------------------
data = {
    "Field": [
        "Invoice Date", "Invoice Number", "Order Number", "Seller Name",
        "Seller Address", "Buyer Name", "Buyer Address", "PAN No",
        "GST Registration No", "Phone Number", "Total Amount", "Tax Amount"
    ],
    "Value": [
        invoice_date, invoice_number, order_number, seller_name,
        seller_address, buyer_name, buyer_address, pan_no,
        gst_no, phone_no, total_amount, tax_amount
    ]
}

df = pd.DataFrame(data)
print("\n=== Extracted Invoice Data ===")
print(df.to_string(index=False))



=== Extracted Invoice Data ===
              Field                  Value
       Invoice Date              Not Found
     Invoice Number       10525RD001820847
       Order Number 1290196-056445-8121802
        Seller Name              Not Found
     Seller Address              Not Found
         Buyer Name              Not Found
      Buyer Address              Not Found
             PAN No              Not Found
GST Registration No              Not Found
       Phone Number              Not Found
       Total Amount              Not Found
         Tax Amount              Not Found


In [23]:
import cv2
import pytesseract
import re
import pandas as pd

# -----------------------------
# 1. Configure Tesseract Path (if needed)
# -----------------------------
# Ensure the Tesseract executable path is correct.
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
custom_config = r'--oem 3 --psm 6'

# -----------------------------
# 2. Read and Preprocess the Invoice Image
# -----------------------------
# Replace with your invoice image path
image_path = r"images\rrr.png"
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"Image not found at {image_path}")

# Convert to grayscale for improved OCR accuracy
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# Optionally, you can apply thresholding:
# gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

# -----------------------------
# 3. Extract Text Using Tesseract OCR
# -----------------------------
ocr_text = pytesseract.image_to_string(gray, config=custom_config)
print("=== OCR Extracted Text ===")
print(ocr_text)

# -----------------------------
# 4. Inline Clean the Text
# -----------------------------
# Remove extra spaces and newlines
cleaned_text = " ".join(ocr_text.split())
print("\n=== Cleaned Text ===")
print(cleaned_text)



=== OCR Extracted Text ===
“_ | | i | | ll |
8300755602

Invoice Number: 10525RD001820847 PackettD: #200755602
Order Number: 1290196-056445-8121802 Tnvoice Date: 31 Jan 2025,
Nature of Transaction: In-State Order Date: 31 Jan 2025
Place of Supply: UTTAR PRADESH Nature of Supply: Goods
Bl to Ship wo:
Anshika Arya
Bit general store, sadapur Noida Sector 45 Gautam Buddha Customer Type: Unregistered
Nagat - 201303 UP, Indl
Bl From: Ship From:
REDTAPE LIMITED REDTAPE LIMITED
C-4,5, 36,37, Sector $9, Nolda 201301 (Utar Pradesh), Nola, C4, 5, 36. 7, Sector 59, Nolda~ 201301 (Uta Prades), Noida,
ar pradesh 201301 ‘ar pradesh- 201301
GSTIN Number: O9AALCRSO22RIZN

Gross Other Taxable sGsT/
ye Discount Clee ENN cost LSE IGST Coss Ttat Amount
RDTPCASH101503720(R1.L0821-39) - Red Tape Women Round Toe Lace-Ups Sneakers, Size: 6 (UK6)
HSN: 64041990, 9.0% CGST, 9.0% SGSTIUGST,
1 Rs 6899.00 Rs 5658.00 _Rs0,00 Rs 1051.70 Rs 94.65 Rs 94.65 Rs 1241.00
TOTAL Rs 6890.00 Rs565800 _Rs0.00 Rs 1051.70 Rs94.65 

In [24]:
 # -----------------------------
# 5. Inline Extraction of Key Invoice Fields
# -----------------------------
# Extract Invoice Number
invoice_number = re.search(r"Invoice Number:\s*([\w-]+)", cleaned_text)
invoice_number = invoice_number.group(1).strip() if invoice_number else "Not Found"

# Extract Invoice Date (accounting for potential misspelling as "Tnvoice Date")
invoice_date = re.search(r"(?:Invoice Date|Tnvoice Date):\s*([\d]{1,2}\s+\w+\s+\d{4})", cleaned_text)
invoice_date = invoice_date.group(1).strip() if invoice_date else "Not Found"

# Extract Order Number
order_number = re.search(r"Order Number:\s*([\w-]+)", cleaned_text)
order_number = order_number.group(1).strip() if order_number else "Not Found"

# Extract Buyer details from the original OCR text using markers
buyer_section = re.search(r"Bl to Ship wo:\s*(.*?)\s*Customer Type:", ocr_text, re.DOTALL)
if buyer_section:
    buyer_lines = buyer_section.group(1).strip().splitlines()
    buyer_name = buyer_lines[0].strip() if buyer_lines else "Not Found"
    buyer_address = " ".join(buyer_lines[1:]).strip() if len(buyer_lines) > 1 else "Not Found"
else:
    buyer_name, buyer_address = "Not Found", "Not Found"

# Extract Seller details from the original OCR text using markers
seller_section = re.search(r"Bl From:\s*Ship From:\s*(.*?)\s*GSTIN Number:", ocr_text, re.DOTALL)
if seller_section:
    seller_lines = seller_section.group(1).strip().splitlines()
    seller_name = seller_lines[0].strip() if seller_lines else "Not Found"
    seller_address = " ".join(seller_lines[1:]).strip() if len(seller_lines) > 1 else "Not Found"
else:
    seller_name, seller_address = "Not Found", "Not Found"

# Extract GSTIN Number
gstin_number = re.search(r"GSTIN Number:\s*([\w]+)", cleaned_text)
gstin_number = gstin_number.group(1).strip() if gstin_number else "Not Found"

# ---- Extract Amount Section ----
# Look for the TOTAL line and capture the amounts following it.
total_section = re.search(r"TOTAL\s+(.*)", cleaned_text)
if total_section:
    total_values = total_section.group(1).strip()
    # Optionally, extract individual amounts (all patterns starting with 'Rs')
    amounts_list = re.findall(r"Rs\s*[\d,]+\.?\d*", total_values)
    total_amounts = ", ".join(amounts_list) if amounts_list else total_values
else:
    total_amounts = "Not Found"

# -----------------------------
# 6. Organize and Display the Extracted Data
# -----------------------------
data = {
    "Field": [
        "Invoice Number", "Invoice Date", "Order Number",
        "Buyer Name", "Buyer Address", "Seller Name", "Seller Address",
        "GSTIN Number", "Total Amounts"
    ],
    "Value": [
        invoice_number, invoice_date, order_number,
        buyer_name, buyer_address, seller_name, seller_address,
        gstin_number, total_amounts
    ]
}

df = pd.DataFrame(data)
print("\n=== Extracted Invoice Data ===")
print(df.to_string(index=False))


=== Extracted Invoice Data ===
         Field                                                                                                                                                         Value
Invoice Number                                                                                                                                              10525RD001820847
  Invoice Date                                                                                                                                                   31 Jan 2025
  Order Number                                                                                                                                        1290196-056445-8121802
    Buyer Name                                                                                                                                                  Anshika Arya
 Buyer Address                                                                                         

In [25]:
## next code 

In [26]:
import cv2
import pytesseract
import re
import pandas as pd

# -----------------------------
# 1. Configure Tesseract Path (if needed)
# -----------------------------
# Set the full path to the Tesseract executable.
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
custom_config = r'--oem 3 --psm 6'

# -----------------------------
# 2. Read and Preprocess the Invoice Image
# -----------------------------
# Replace with the path to your invoice image
image_path = r"images\web.webp"  # <-- Update this path
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"Image not found at {image_path}")

# Convert to grayscale to improve OCR accuracy
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# Optionally, apply thresholding:
# gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

# (Optional) Save the preprocessed image for debugging
cv2.imwrite("preprocessed_image.jpg", gray)

# -----------------------------
# 3. Extract Text Using Tesseract OCR
# -----------------------------
ocr_text = pytesseract.image_to_string(gray, config=custom_config)
print("=== OCR Extracted Text ===")
print(ocr_text)

# -----------------------------
# 4. Clean the Extracted Text
# -----------------------------
# Remove extra spaces and newline characters
cleaned_text = " ".join(ocr_text.split())
print("\n=== Cleaned Text ===")
print(cleaned_text)


=== OCR Extracted Text ===
amazon.in Tax Invoice/Bill of Supply/Cash Memo
Se (Original for Recipient)
Signature Not Verlted @
Dalal sgredn 2s cLouDTAL gd Prvare LnITED 1
Reason: involce a
Sold By: ” .
Clouatail India Private Limited collins Addtoes:
Aruna Warehousing Company, Shed No. B2 & ,
‘A2,Sy No 14 and 170, Somaram Village, Medchal Mig 6 ater acer Neer SaRRTOIG caical
Mandal, Rangareddy Dist
Meee ae eso 1aot Hyderabad, AndhraPradesh, 500038
IN
PAN No: AAQCS4259Q Shipping Address:
GST Registration No: 36AAQCS425901ZB Chandra Bhushan
Chandra Bhushan
Mig 6 Ground Floor Uday Nagar Colony, S R
Nagar Near Hindu Public School
Hyderabad, AndhraPradesh, 500038
IN
‘Order Number: 407-3225044-9079525 Invoice Number: SHYB-10719
Order Date: 26.04.2018 Invoice Details: TG-SHYB-1004-1819
Invoice Date: 27.04.2018
fiefpeserouon price lontamount [pate [amount moun |
IWhirpoo' 900_L Frost Free Mull-Door Refigeraor (FP
NsNeaie
u%s)sast].389.53
Fora, Re r7006 | O;G HU.
[Amount in Words:
[Thirty Thousa

In [27]:
# -----------------------------
# 5. Inline Extraction of Key Invoice Fields for an Indian Invoice
# -----------------------------

# Invoice Number (e.g., "Invoice Number: INV12345")
invoice_number = re.search(r"Invoice Number:\s*([\w-]+)", cleaned_text)
invoice_number = invoice_number.group(1).strip() if invoice_number else "Not Found"

# Invoice Date (formats like "31 Jan 2025", "31/01/2025", or "31-01-2025")
invoice_date = re.search(r"Invoice Date:\s*([\d]{1,2}[\-/\s]\w+[\-/\s]\d{4})", cleaned_text)
invoice_date = invoice_date.group(1).strip() if invoice_date else "Not Found"

# Order Number
order_number = re.search(r"Order Number:\s*([\w-]+)", cleaned_text)
order_number = order_number.group(1).strip() if order_number else "Not Found"

# Seller Details: Look for labels like "Sold By" or "Seller Name" and "Seller Address"
seller_name = re.search(r"(?:Seller(?:'s)? Name|Sold By):\s*([^,]+)", cleaned_text)
seller_name = seller_name.group(1).strip() if seller_name else "Not Found"

seller_address = re.search(r"Seller Address:\s*(.*?)\s*(?:GSTIN|PAN)", cleaned_text, re.DOTALL)
seller_address = seller_address.group(1).strip() if seller_address else "Not Found"

# Buyer Details: Look for labels like "Bill To" or "Buyer Name" and "Buyer Address"
buyer_name = re.search(r"(?:Bill To|Buyer(?:'s)? Name):\s*([^,]+)", cleaned_text)
buyer_name = buyer_name.group(1).strip() if buyer_name else "Not Found"

buyer_address = re.search(r"Buyer Address:\s*(.*?)\s*(?:GSTIN|PAN)", cleaned_text, re.DOTALL)
buyer_address = buyer_address.group(1).strip() if buyer_address else "Not Found"

# GSTIN (for seller, generally)
gstin = re.search(r"GST(?:IN)?(?: Number)?:\s*([\w]+)", cleaned_text)
gstin = gstin.group(1).strip() if gstin else "Not Found"

# PAN (if available)
pan = re.search(r"PAN(?: Number)?:\s*([\w]+)", cleaned_text)
pan = pan.group(1).strip() if pan else "Not Found"

# Total Amount (e.g., "Total Amount: Rs 12,345.67")
total_amount = re.search(r"Total Amount:\s*Rs\.?\s*([\d,]+\.?\d*)", cleaned_text)
total_amount = total_amount.group(1).strip() if total_amount else "Not Found"

# Tax Details (if available)
cgst = re.search(r"CGST:\s*Rs\.?\s*([\d,]+\.?\d*)", cleaned_text)
cgst = cgst.group(1).strip() if cgst else "Not Found"

sgst = re.search(r"SGST:\s*Rs\.?\s*([\d,]+\.?\d*)", cleaned_text)
sgst = sgst.group(1).strip() if sgst else "Not Found"

igst = re.search(r"IGST:\s*Rs\.?\s*([\d,]+\.?\d*)", cleaned_text)
igst = igst.group(1).strip() if igst else "Not Found"

# -----------------------------
# 6. Organize and Display the Extracted Data in a Neat Format
# -----------------------------
data = {
    "Field": [
        "Invoice Number", "Invoice Date", "Order Number",
        "Seller Name", "Seller Address", "Buyer Name", "Buyer Address",
        "GSTIN", "PAN", "Total Amount", "CGST", "SGST", "IGST"
    ],
    "Value": [
        invoice_number, invoice_date, order_number,
        seller_name, seller_address, buyer_name, buyer_address,
        gstin, pan, total_amount, cgst, sgst, igst
    ]
}

df = pd.DataFrame(data)
print("\n=== Extracted Invoice Data ===")
print(df.to_string(index=False))


=== Extracted Invoice Data ===
         Field                                                                          Value
Invoice Number                                                                     SHYB-10719
  Invoice Date                                                                      Not Found
  Order Number                                                            407-3225044-9079525
   Seller Name ” . Clouatail India Private Limited collins Addtoes: Aruna Warehousing Company
Seller Address                                                                      Not Found
    Buyer Name                                                                      Not Found
 Buyer Address                                                                      Not Found
         GSTIN                                                                      Not Found
           PAN                                                                      Not Found
  Total Amount              